In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from feature_engine.outliers import Winsorizer
from sklearn.model_selection import KFold, cross_validate

In [2]:
# Load the classification dataset
dft = pd.read_csv(r"C:\Users\basde\Documents\GitHub\Code-and-examples\Projects\Binary Classification with a Bank Dataset\train.csv")
dfo = pd.read_csv(r"C:\Users\basde\Documents\GitHub\Code-and-examples\Projects\Binary Classification with a Bank Dataset\bank-full.csv", delimiter=';')
dfo['y'] = dfo['y'].map({'no' : 0, 'yes': 1})
# df = pd.concat([dft, dfo])
y = dfo['y']
dfo = dfo.drop('y', axis=1)

In [3]:
dfo['default'] = pd.get_dummies(dfo['default'], drop_first=True, dtype=int)
dfo['housing'] = pd.get_dummies(dfo['housing'], drop_first=True, dtype=int)
dfo['loan'] = pd.get_dummies(dfo['loan'], drop_first=True, dtype=int)
dfo['contact'] = dfo['contact'].map({'telephone': 'cellular', 'unknown' : 'unknown', 'telephone': 'telephone'})
dfo['contact'] = pd.get_dummies(dfo['contact'], drop_first=True, dtype=int)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,58,management,married,tertiary,0,2143,1,0,1,5,may,261,1,-1,0,unknown
1,44,technician,single,secondary,0,29,1,0,1,5,may,151,1,-1,0,unknown
2,33,entrepreneur,married,secondary,0,2,1,1,1,5,may,76,1,-1,0,unknown
3,47,blue-collar,married,unknown,0,1506,1,0,1,5,may,92,1,-1,0,unknown
4,33,unknown,single,unknown,0,1,0,0,1,5,may,198,1,-1,0,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,0,825,0,0,0,17,nov,977,3,-1,0,unknown
45207,71,retired,divorced,primary,0,1729,0,0,0,17,nov,456,2,-1,0,unknown
45208,72,retired,married,secondary,0,5715,0,0,0,17,nov,1127,5,184,3,success
45209,57,blue-collar,married,secondary,0,668,0,0,0,17,nov,508,4,-1,0,unknown


In [8]:
X = dfo.select_dtypes(include=np.number)

In [9]:
# a. Specify the feature to apply transformations on
# You can change 'Age' to any other numerical column name from your dataframe
feature_to_transform = 'duration'

# b. Define the classification models to evaluate
models = {
    "Logistic Regression": LogisticRegression(max_iter=200, random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Support Vector Machine": SVC(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, eval_metric='logloss')
}

# c. Define the transformation pipelines to test on the specified feature
# The ColumnTransformer applies the specified pipeline to the 'feature_to_transform'
# and leaves the other features untouched (remainder='passthrough').
transformation_pipelines = {
    'Log Transformation': ColumnTransformer(
        transformers=[
            ('transformer', Pipeline([
                ('imputer', SimpleImputer(strategy='median')),
                ('log', FunctionTransformer(np.log1p))
            ]), [feature_to_transform])
        ],
        remainder='passthrough'
    ),
    'Square Root Transformation': ColumnTransformer(
        transformers=[
            ('transformer', Pipeline([
                ('imputer', SimpleImputer(strategy='median')),
                ('sqrt', FunctionTransformer(np.sqrt))
            ]), [feature_to_transform])
        ],
        remainder='passthrough'
    ),
    'Winsorization (0-95%)': ColumnTransformer(
        transformers=[
            ('transformer', Pipeline([
                ('imputer', SimpleImputer(strategy='median')),
                ('winsorizer', Winsorizer(capping_method='quantiles', tail='right', fold=0.05))
            ]), [feature_to_transform])
        ],
        remainder='passthrough'
    )
}

# d. Define the classification scoring metrics
scoring_metrics = {
    'accuracy': 'accuracy',
    'f1_score': 'f1_weighted',
    'precision': 'precision_weighted',
    'recall': 'recall_weighted'
}

# e. Define the cross-validation strategy
cv_strategy = KFold(n_splits=5, shuffle=True, random_state=42)

In [10]:
# This DataFrame will hold all results for final comparison
all_results = pd.DataFrame()

# --- Main Loop ---
for model_name, model in models.items():
    print(f"--- Evaluating Model: {model_name} ---")
    model_results = {}

    # a. "Feature Dropped" Evaluation
    # Dynamically drops the feature specified in cell 3
    X_dropped = X.drop(columns=[feature_to_transform])
    feature_dropped_scores = cross_validate(
        model, X_dropped, y, cv=cv_strategy,
        scoring=scoring_metrics, return_train_score=True
    )
    model_results['Feature Dropped'] = {
        'Train F1-Score': feature_dropped_scores['train_f1_score'].mean(),
        'CV F1-Score': feature_dropped_scores['test_f1_score'].mean(),
        'CV Accuracy': feature_dropped_scores['test_accuracy'].mean()
    }
    print(f"--- feature dropped: {model_name} ---")
    # b. "Imputation Only" Baseline Evaluation
    # Applies imputation only to the specified feature, others are passed through
    baseline_preprocessor = ColumnTransformer(
        transformers=[
            ('imputer', SimpleImputer(strategy='median'), [feature_to_transform])
        ],
        remainder='passthrough'
    )
    baseline_pipeline = Pipeline(steps=[
        ('preprocessor', baseline_preprocessor),
        ('model', model)
    ])
    baseline_scores = cross_validate(
        baseline_pipeline, X, y, cv=cv_strategy,
        scoring=scoring_metrics, return_train_score=True
    )
    model_results['Baseline (Imputation Only)'] = {
        'Train F1-Score': baseline_scores['train_f1_score'].mean(),
        'CV F1-Score': baseline_scores['test_f1_score'].mean(),
        'CV Accuracy': baseline_scores['test_accuracy'].mean()
    }
    print(f"--- imputation only: {model_name} ---")
    # c. Transformation Pipelines Evaluation
    # Iterates through the ColumnTransformer pipelines defined in cell 3
    for tech_name, preprocessor in transformation_pipelines.items():
        full_pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', model)
        ])
        scores = cross_validate(
            full_pipeline, X, y, cv=cv_strategy,
            scoring=scoring_metrics, return_train_score=True
        )
        model_results[tech_name] = {
            'Train F1-Score': scores['train_f1_score'].mean(),
            'CV F1-Score': scores['test_f1_score'].mean(),
            'CV Accuracy': scores['test_accuracy'].mean()
        }
        print(f"--- tranformation: {tech_name} ---")
    # d. Consolidate and store results
    temp_df = pd.DataFrame.from_dict(model_results, orient='index')
    temp_df['Model'] = model_name
    all_results = pd.concat([all_results, temp_df])

# e. Final processing for the results table
all_results.reset_index(inplace=True)
all_results.rename(columns={'index': 'Preprocessing Technique'}, inplace=True)
all_results['Generalization'] = all_results['CV F1-Score'] / all_results['Train F1-Score']
all_results = all_results.sort_values(by='CV F1-Score', ascending=False)


--- Evaluating Model: Logistic Regression ---


C:\Users\basde\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 200 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=200).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\basde\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 200 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=200).
You might also want to scale the data as shown in:
    https://sciki

--- feature dropped: Logistic Regression ---


C:\Users\basde\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 200 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=200).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\basde\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 200 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=200).
You might also want to scale the data as shown in:
    https://sciki

--- imputation only: Logistic Regression ---


...

--- tranformation: Log Transformation ---


C:\Users\basde\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:23:38] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\basde\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:23:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\basde\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:23:40] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\basde\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:23:40] WARNING: C:\actio

--- tranformation: Square Root Transformation ---


C:\Users\basde\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:23:42] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\basde\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:23:43] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\basde\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:23:44] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\basde\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:23:45] WARNING: C:\actio

--- tranformation: Winsorization (0-95%) ---


In [11]:
# Reorder columns for a more logical presentation
final_columns_order = [
    'Model',
    'Preprocessing Technique',
    'CV F1-Score',
    'CV Accuracy',
    'Train F1-Score',
    'Generalization'
]
all_results = all_results[final_columns_order]

all_results

,Model,Preprocessing Technique,CV F1-Score,CV Accuracy,Train F1-Score,Generalization
16,XGBoost,Baseline (Imputation Only),0.888526,0.897525,0.933883,0.951432
17,XGBoost,Log Transformation,0.888526,0.897525,0.933883,0.951432
18,XGBoost,Square Root Transformation,0.888526,0.897525,0.933883,0.951432
19,XGBoost,Winsorization (0-95%),0.886795,0.895756,0.934204,0.949252
7,Random Forest,Log Transformation,0.886537,0.897038,0.999983,0.886552
8,Random Forest,Square Root Transformation,0.886471,0.896972,0.999983,0.886485
6,Random Forest,Baseline (Imputation Only),0.886368,0.896928,0.999983,0.886383
9,Random Forest,Winsorization (0-95%),0.884286,0.895092,0.999972,0.884310
4,Logistic Regression,Winsorization (0-95%),0.867624,0.886466,0.867535,1.000103
1,Logistic Regression,Baseline (Imputation Only),0.866901,0.889850,0.866592,1.000357
